In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [11]:
class MyTreeClf:
    def __init__(self, max_depth=5, min_samples_split=2, max_leafs=20, bins=None, criterion='entropy'):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_leafs = max_leafs
        self.leafs_cnt = 0  # Переменная для отслеживания количества листьев
        self.tree = None  # Дерево будем хранить здесь
        self.bins = bins
        self.bin_edges = {}  # Словарь для хранения границ бинов для каждой фичи
        self.criterion = criterion  # Новый параметр для выбора критерия
        self.fi = {}  # Словарь для хранения важности фичей
    
    def __str__(self):
        params = vars(self) # Получаем все атрибуты экземпляра как словарь
        params_str = ', '.join(f"{key}={value}" for key, value in params.items())
        return f"MyTreeClf class: {params_str}"
    
    def _initialize_bins(self, X):
        for col in X.columns:
            if self.bins is not None:
                unique_values = np.unique(X[col])
                if len(unique_values) <= self.bins - 1:
                    self.bin_edges[col] = unique_values
                else:
                    hist, edges = np.histogram(X[col], bins=self.bins)
                    # Используем только внутренние границы
                    self.bin_edges[col] = edges[1:-1]
                    
       # Вычисление неопределенности Джини
    def _gini(self, y):
        value_counts = np.bincount(y)  # Подсчет количества каждого класса
        probabilities = value_counts / len(y)  # Вероятности классов
        gini = 1 - np.sum(probabilities ** 2)  # Формула Джини
        return gini
    
    # Вычисление энтропии
    def _entropy(self, y):
        value_counts = np.bincount(y)  # Подсчёт количества каждого класса
        probabilities = value_counts / len(y)  # Вероятности классов
        # Для обработки логарифмов от 0 используем условие (если вероятность 0, то член равен 0)
        entropy = -np.sum([p * np.log2(p) for p in probabilities if p > 0])
        return entropy

    def get_best_split(self, X: pd.DataFrame, y: pd.Series):
        best_split = {
            "col_name": None,
            "split_value": None,
            "ig": -1  # Information Gain (Прирост информации)
        }
        
        # Вычисление базовой неопределенности в зависимости от критерия
        if self.criterion == 'entropy':
            base_uncertainty = self._entropy(y)
        elif self.criterion == 'gini':
            base_uncertainty = self._gini(y)

        # Перебор всех колонок (фичей)
        for col in X.columns:
            # Получаем разделители для данной фичи
            if col in self.bin_edges:
                unique_values = self.bin_edges[col]  # Используем границы бинов
            else:
                unique_values = np.sort(X[col].unique())  # Используем уникальные значения

            # Перебор возможных значений для разделения
            for i in range(1, len(unique_values)):
                split_value = (unique_values[i - 1] + unique_values[i]) / 2  # Разделитель как среднее между двумя значениями

                # Левая и правая подвыборки
                left_mask = X[col] <= split_value
                right_mask = X[col] > split_value

                left_y, right_y = y[left_mask], y[right_mask]

                # Пропускаем разбиение, если одна из подвыборок пуста
                if len(left_y) == 0 or len(right_y) == 0:
                    continue

                # Вычисляем неопределенность для левой и правой подвыборки
                if self.criterion == 'entropy':
                    left_uncertainty = self._entropy(left_y)
                    right_uncertainty = self._entropy(right_y)
                elif self.criterion == 'gini':
                    left_uncertainty = self._gini(left_y)
                    right_uncertainty = self._gini(right_y)

                # Взвешенная неопределенность
                weighted_uncertainty = (len(left_y) / len(y)) * left_uncertainty + (len(right_y) / len(y)) * right_uncertainty

                # Прирост информации (Information Gain)
                information_gain = base_uncertainty - weighted_uncertainty

                # Сравнение с текущим лучшим разделителем
                if information_gain > best_split["ig"]:
                    best_split["col_name"] = col
                    best_split["split_value"] = split_value
                    best_split["ig"] = information_gain

            return best_split["col_name"], best_split["split_value"], best_split["ig"]
    
     # Метод fit для построения дерева
    def fit(self, X: pd.DataFrame, y: pd.Series):
        self.leafs_cnt = 0  # Сброс счетчика листьев
        self.fi = {col: 0 for col in X.columns}  # Инициализация важности фичей
        
         # Инициализация разделителей
        self._initialize_bins(X)

        # Вложенная рекурсивная функция для построения дерева
        def build_tree(X, y, depth=0):
            # Условия остановки
            if len(np.unique(y)) == 1:  # Все элементы одного класса
                self.leafs_cnt += 1
                return np.unique(y)[0]
            
            if depth >= self.max_depth or len(y) < self.min_samples_split or self.leafs_cnt >= self.max_leafs:
                self.leafs_cnt += 1
                return np.bincount(y).argmax()  # Возвращаем наиболее частый класс

            # Поиск лучшего разбиения
            col_name, split_value, ig = self.get_best_split(X, y)

            if ig == -1 or self.leafs_cnt >= self.max_leafs:  # Если прироста информации нет или лимит листьев достигнут, создаём новый лист
                self.leafs_cnt += 1
                return np.bincount(y).argmax()
            
            self.fi[col_name] += ig  # Увеличиваем важность на величину прироста информации

            # Разделение данных
            left_mask = X[col_name] <= split_value
            right_mask = X[col_name] > split_value

            left_tree = build_tree(X[left_mask], y[left_mask], depth + 1)
            right_tree = build_tree(X[right_mask], y[right_mask], depth + 1)

            # Структура узла
            return {
                "col_name": col_name,
                "split_value": split_value,
                "left": left_tree,
                "right": right_tree
            }

        # Строим дерево
        self.tree = build_tree(X, y)
        
    def feature_importances(self):
        return self.fi
        
     # Метод для печати дерева
    def print_tree(self, node=None, depth=0):
        if node is None:
            node = self.tree
        
        # Если это лист (значение), выводим его
        if not isinstance(node, dict):
            print(f"{'  ' * depth}leaf = {node}")
            return
        
        # Если это узел, выводим информацию о разбиении
        print(f"{'  ' * depth}{node['col_name']} > {node['split_value']}")
        
        # Рекурсивно печатаем левую и правую ветви
        print(f"{'  ' * (depth + 1)}Left:")
        self.print_tree(node['left'], depth + 2)
        
        print(f"{'  ' * (depth + 1)}Right:")
        self.print_tree(node['right'], depth + 2)
        
    # Метод для предсказания вероятностей
    def predict_proba(self, X: pd.DataFrame):
        probabilities = []
        
        for _, row in X.iterrows():
            probabilities.append(self._predict_proba_single(row, self.tree))
        
        return np.array(probabilities)

    # Вспомогательный метод для предсказания одной строки
    def _predict_proba_single(self, row, node):
        # Если это лист (значение), возвращаем вероятность
        if not isinstance(node, dict):
            return node

        # Переход к следующему узлу
        col_name = node['col_name']
        split_value = node['split_value']

        if row[col_name] <= split_value:
            return self._predict_proba_single(row, node['left'])
        else:
            return self._predict_proba_single(row, node['right'])

    # Метод для предсказания классов
    def predict(self, X: pd.DataFrame):
        probabilities = self.predict_proba(X)
        return (probabilities > 0.5).astype(int)


In [12]:
# Важность фичей
X = pd.DataFrame({
    'feature1': [1, 2, 3, 4, 5],
    'feature2': [5, 4, 3, 2, 1],
    'feature3': [1, 3, 5, 7, 9]
})

y = pd.Series([0, 1, 0, 1, 0])

# Инициализация и обучение модели
model = MyTreeClf(max_depth=3, criterion='entropy')
model.fit(X, y)

# Получение важности фичей
importances = model.feature_importances()
print(importances)


{'feature1': 0.7338578863016243, 'feature2': 0, 'feature3': 0}


In [10]:
#Тестирование предсказания
X, y = make_classification(n_samples=100, n_features=5, n_informative=3, n_redundant=0, random_state=42)
X = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
y = pd.Series(y)

my_clf1 = MyTreeClf(max_depth=3, min_samples_split=2, max_leafs=10, criterion='entropy')
my_clf1.fit(X, y)
my_clf2 = MyTreeClf(max_depth=4, min_samples_split=3, max_leafs=30, criterion='gini')
my_clf2.fit(X, y)

# Получение вероятностей и предсказаний
my_probabilities1 = my_clf1.predict_proba(X)
my_predictions1 = my_clf1.predict(X)
my_probabilities2 = my_clf2.predict_proba(X)
my_predictions2 = my_clf2.predict(X)

# Сумма вероятностей
sum_my_clf1 = np.sum(my_predictions1) + np.sum(my_probabilities1)
sum_my_clf2 = np.sum(my_predictions2) + np.sum(my_probabilities2)

print(f"Сумма: {sum_my_clf1} и {sum_my_clf2}")

Сумма: 104 и 100


In [4]:
#Тестирование обучения
X, y = make_classification(n_samples=100, n_features=5, n_informative=3, n_redundant=2, random_state=42)

# Преобразуем в DataFrame и Series
X_test = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
y_test = pd.Series(y)

# Создаем объект класса
clf = MyTreeClf(max_depth=3, min_samples_split=2, max_leafs=6)

# Вызываем метод fit для теста
clf.fit(X_test, y_test)

# Печатаем дерево
print("Построенное дерево:")
clf.print_tree()

# Проверяем количество листьев
print("\nКоличество листьев:")
print(clf.leafs_cnt)

Построенное дерево:
feature_3 > 0.10692195695825762
  Left:
    feature_2 > 0.5037529278758279
      Left:
        feature_3 > -0.4069952905845129
          Left:
            leaf = 0
          Right:
            leaf = 0
      Right:
        leaf = 1
  Right:
    feature_1 > -0.6408632697678901
      Left:
        feature_3 > 1.4043361004297479
          Left:
            leaf = 0
          Right:
            leaf = 1
      Right:
        feature_3 > 0.5167263246823051
          Left:
            leaf = 1
          Right:
            leaf = 1

Количество листьев:
7


In [12]:
#Тестирование поиска наилучшего сплита
# Создаем тестовый набор данных
X, y = make_classification(n_samples=100, n_features=5, n_informative=3, random_state=42)

# Преобразуем X в DataFrame для удобства работы с нашим методом
X_df = pd.DataFrame(X, columns=[f'col_{i}' for i in range(X.shape[1])])

# Разделение на тренировочную и тестовую выборки для проверки
X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.2, random_state=42)

# Инициализируем наш класс
clf = MyTreeClf()

# Применяем метод get_best_split на тренировочных данных
best_split = clf.get_best_split(X_train, y_train)

# Выводим результат
print("Лучший сплит:")
print(f"Фича: {best_split[0]}")
print(f"Значение для разделения: {best_split[1]}")
print(f"Прирост информации: {best_split[2]}")

Лучший сплит:
Фича: col_3
Значение для разделения: 0.20389423213510222
Прирост информации: 0.5720798775568654


In [5]:
# Тестирование класса
tree1 = MyTreeClf()
tree2 = MyTreeClf(max_depth = 10, max_leafs = 30)
tree3 = MyTreeClf(max_leafs = 25, min_samples_split = 15)

# Проверка
print(tree1)
print(tree2)
print(tree3)

MyTreeClf class: max_depth=5, min_samples_split=2, max_leafs=20
MyTreeClf class: max_depth=10, min_samples_split=2, max_leafs=30
MyTreeClf class: max_depth=5, min_samples_split=15, max_leafs=25
